In [2]:
import csv
import pandas as pd

## Player Analysis

In [88]:
player_df = pd.read_csv("csv/draf.csv")

### Origin
Country, school or the state (if the player is a US citizen)

In [100]:
player_df[player_df['last_name']=='Curry']


,person_id,first_name,last_name,display_first_last,display_last_comma_first,display_fi_last,player_slug,birthdate,school,country,...,playercode,from_year,to_year,dleague_flag,nba_flag,games_played_flag,draft_year,draft_round,draft_number,greatest_75_flag
221,203552,Seth,Curry,Seth Curry,"Curry, Seth",S. Curry,seth-curry,1990-08-23 00:00:00,Duke,USA,...,seth_curry,2013.0,2022.0,Y,Y,Y,Undrafted,Undrafted,Undrafted,N


In [101]:
player_df.columns

Index(['person_id', 'first_name', 'last_name', 'display_first_last',
       'display_last_comma_first', 'display_fi_last', 'player_slug',
       'birthdate', 'school', 'country', 'last_affiliation', 'height',
       'weight', 'season_exp', 'jersey', 'position', 'rosterstatus',
       'games_played_current_season_flag', 'team_id', 'team_name',
       'team_abbreviation', 'team_code', 'team_city', 'playercode',
       'from_year', 'to_year', 'dleague_flag', 'nba_flag', 'games_played_flag',
       'draft_year', 'draft_round', 'draft_number', 'greatest_75_flag'],
      dtype='object')

In [90]:
len(player_df.index)

2457